In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.optimizers import Adadelta, SGD, RMSprop, Adagrad, Adam, Adamax

from keras.regularizers import l1_l2
from keras.callbacks import EarlyStopping, Callback
from sklearn.base import BaseEstimator, ClassifierMixin
from keras import backend as K
import numpy as np
import logging
from keras.layers import Activation, Dense
    

In [5]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler as SS
from sklearn.cross_validation import cross_val_score

In [6]:
from __future__ import print_function
import keras
from keras.datasets import cifar10

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [8]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
x_train = x_train.reshape(50000, 3072)
x_test = x_test.reshape(10000, 3072)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
#Splitting dataset into training, validation and testing 
from sklearn.cross_validation import train_test_split
X_trn, X_val, y_trn, y_val = train_test_split(x_train, y_train, test_size=0.2,)


In [11]:
num_classes=10
#y_trn = keras.utils.to_categorical(y_trn, num_classes)
#y_val = keras.utils.to_categorical(y_val, num_classes)
y_train=keras.utils.to_categorical(y_train, num_classes)


In [12]:
y_test = keras.utils.to_categorical(y_test, num_classes)

In [13]:
y_val = keras.utils.to_categorical(y_val, num_classes)

In [14]:
y_trn = keras.utils.to_categorical(y_trn, num_classes)

In [15]:
#Model 1
from keras import optimizers

# All parameter gradients will be clipped to
# a maximum value of 0.5 and
# a minimum value of -0.5.
sgd = optimizers.SGD(lr=0.06, clipvalue=0.5,momentum=0.03, decay=0.01, nesterov=False)

adam=keras.optimizers.Adamax(lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model1 = Sequential()
model1.add(Dense(512, activation='relu', input_shape=(3072,)))
model1.add(Dropout(0.2))
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.2))

model1.add(Dense(10, activation='softmax'))

model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total para

In [12]:
model1.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [13]:
history1 = model1.fit(X_trn, y_trn,
                    batch_size=32,
                    epochs=30,
                    verbose=1,
                    validation_data=(X_val, y_val))

Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 17s - loss: 1.9328 - acc: 0.2977 - val_loss: 1.7938 - val_acc: 0.3625
Epoch 2/30
40000/40000 [==============================] - 16s - loss: 1.8022 - acc: 0.3518 - val_loss: 1.7432 - val_acc: 0.3806
Epoch 3/30
40000/40000 [==============================] - 16s - loss: 1.7706 - acc: 0.3621 - val_loss: 1.7171 - val_acc: 0.3891
Epoch 4/30
40000/40000 [==============================] - 16s - loss: 1.7433 - acc: 0.3735 - val_loss: 1.6984 - val_acc: 0.3940
Epoch 5/30
40000/40000 [==============================] - 17s - loss: 1.7353 - acc: 0.3761 - val_loss: 1.6886 - val_acc: 0.3974
Epoch 6/30
40000/40000 [==============================] - 16s - loss: 1.7196 - acc: 0.3842 - val_loss: 1.6764 - val_acc: 0.4012
Epoch 7/30
40000/40000 [==============================] - 16s - loss: 1.7132 - acc: 0.3871 - val_loss: 1.6694 - val_acc: 0.4028
Epoch 8/30
40000/40000 [==============================

In [15]:
import json


In [16]:
train_losses, train_accs = [], []
val_losses, val_accs = [], []

In [43]:
for e in range(30):

        loss = model1.fit(X_trn, y_trn,
                         batch_size=32,
                         validation_data=(x_test, y_test),
                         nb_epoch=30)

        train_losses.append(loss.history["loss"])
        val_losses.append(loss.history["val_loss"])
        train_accs.append(loss.history["acc"])
        val_accs.append(loss.history["val_acc"])

        # Save experimental log
        d_log = {}
        d_log["experiment_name"] = "Model1"
        d_log["img_dim"] = 32*32*3
        d_log["batch_size"] = 32
        d_log["nb_epoch"] = 30
        d_log["train_losses"] = train_losses
        d_log["val_losses"] = val_losses
        d_log["train_accs"] = train_accs
        d_log["val_accs"] = val_accs
        d_log["optimizer"] = SGD
        # Add model architecture
        json_string = json.loads(model1.to_json())
        for key in json_string.keys():
            d_log[key] = json_string[key]
        list_color = [u'#E24A33',
                  u'#348ABD',
                  u'#FBC15E',
                  u'#777777',
                  u'#988ED5',
                  u'#8EBA42',
                  u'#FFB5B8']
        plt.figure()
        plt.ylabel(to_plot, fontsize=20)
        if to_plot == "losses":
            plt.yscale("log")
        if to_plot == "accs":
            plt.ylim([0, 1.1])
        plt.xlabel("Number of epochs", fontsize=20)
        plt.title("%s experiment" % dataset, fontsize=22)
        plt.legend(loc="best")
        plt.tight_layout()
        plt.savefig("./figures/Model1_results.png" )
        plt.show()

Train on 40000 samples, validate on 10000 samples
Epoch 1/30
  512/40000 [..............................] - ETA: 15s - loss: 1.5433 - acc: 0.4570

/usr/local/src/conda3_runtime.v18/4.1.1/lib/python3.5/site-packages/keras/models.py:851: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


40000/40000 [==============================] - 17s - loss: 1.5845 - acc: 0.4340 - val_loss: 1.5407 - val_acc: 0.4522
Epoch 2/30
40000/40000 [==============================] - 16s - loss: 1.5851 - acc: 0.4340 - val_loss: 1.5399 - val_acc: 0.4527
Epoch 3/30
40000/40000 [==============================] - 17s - loss: 1.5814 - acc: 0.4384 - val_loss: 1.5402 - val_acc: 0.4530
Epoch 4/30
40000/40000 [==============================] - 16s - loss: 1.5853 - acc: 0.4375 - val_loss: 1.5393 - val_acc: 0.4523
Epoch 5/30
40000/40000 [==============================] - 16s - loss: 1.5841 - acc: 0.4336 - val_loss: 1.5396 - val_acc: 0.4528
Epoch 6/30
40000/40000 [==============================] - 16s - loss: 1.5826 - acc: 0.4373 - val_loss: 1.5392 - val_acc: 0.4526
Epoch 7/30
40000/40000 [==============================] - 16s - loss: 1.5839 - acc: 0.4368 - val_loss: 1.5388 - val_acc: 0.4533
Epoch 8/30
40000/40000 [==============================] - 16s - loss: 1.5822 - acc: 0.4360 - val_loss: 1.5386 - val

RuntimeError: Invalid DISPLAY variable

In [44]:
score1 = model1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

Test loss: 1.53350017071
Test accuracy: 0.4559


In [45]:
#Model 2
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              3146752   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
__________

In [46]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#RMSprop()

In [47]:
history = model.fit(X_trn, y_trn,
                    batch_size=25,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_val, y_val))

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 32s - loss: 2.3833 - acc: 0.0976 - val_loss: 2.3064 - val_acc: 0.0969
Epoch 2/20
40000/40000 [==============================] - 31s - loss: 2.3525 - acc: 0.1021 - val_loss: 2.2914 - val_acc: 0.1339
Epoch 3/20
40000/40000 [==============================] - 31s - loss: 2.3321 - acc: 0.1083 - val_loss: 2.2827 - val_acc: 0.1343
Epoch 4/20
40000/40000 [==============================] - 31s - loss: 2.3237 - acc: 0.1126 - val_loss: 2.2758 - val_acc: 0.1284
Epoch 5/20
40000/40000 [==============================] - 31s - loss: 2.3141 - acc: 0.1137 - val_loss: 2.2699 - val_acc: 0.1278
Epoch 6/20
40000/40000 [==============================] - 31s - loss: 2.3065 - acc: 0.1195 - val_loss: 2.2644 - val_acc: 0.1274
Epoch 7/20
40000/40000 [==============================] - 30s - loss: 2.2995 - acc: 0.1264 - val_loss: 2.2593 - val_acc: 0.1301
Epoch 8/20
40000/40000 [==============================

In [48]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.18801478119
Test accuracy: 0.2002


In [49]:
#Model 3
model2 = Sequential()
model2.add(Dense(512, activation='relu', input_shape=(3072,)))
model2.add(Dropout(0.2))
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(512, activation='tanh'))

model2.add(Dense(10, activation='softmax'))

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               1573376   
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 2,103,818
Trainable params: 2,103,818
Non-trainable params: 0
_________________________________________________________________


In [51]:
model2.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
history2 = model2.fit(X_trn, y_trn,
                    batch_size=128,
                    epochs=40,
                    verbose=1,
                    validation_data=(X_val, y_val))

Train on 40000 samples, validate on 10000 samples
Epoch 1/40
40000/40000 [==============================] - 6s - loss: 2.4415 - acc: 0.0950 - val_loss: 2.3824 - val_acc: 0.0958
Epoch 2/40
40000/40000 [==============================] - 5s - loss: 2.3974 - acc: 0.0996 - val_loss: 2.3465 - val_acc: 0.0966
Epoch 3/40
40000/40000 [==============================] - 5s - loss: 2.3681 - acc: 0.1003 - val_loss: 2.3235 - val_acc: 0.1046
Epoch 4/40
40000/40000 [==============================] - 5s - loss: 2.3527 - acc: 0.1045 - val_loss: 2.3076 - val_acc: 0.1197
Epoch 5/40
40000/40000 [==============================] - 5s - loss: 2.3364 - acc: 0.1135 - val_loss: 2.2959 - val_acc: 0.1328
Epoch 6/40
40000/40000 [==============================] - 5s - loss: 2.3277 - acc: 0.1152 - val_loss: 2.2870 - val_acc: 0.1377
Epoch 7/40
40000/40000 [==============================] - 5s - loss: 2.3207 - acc: 0.1175 - val_loss: 2.2800 - val_acc: 0.1444
Epoch 8/40
40000/40000 [==============================] - 5s 

In [52]:
score2 = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

Test loss: 2.17451163864
Test accuracy: 0.2344


In [16]:
#Model 4
from keras import regularizers
model3 = Sequential()
model3.add(Dense(512, activation='relu', input_shape=(3072,),kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model3.add(Dropout(0.2))
model3.add(Dense(512, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(512, activation='tanh'))

model3.add(Dense(10, activation='softmax'))

model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                5130      
Total params: 2,103,818
Trainable params: 2,103,818
Non-trainable params: 0
_________________________________________________________________


In [17]:
model3.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [ ]:
history3 = model3.fit(X_trn, y_trn,
                    batch_size=30,
                    epochs=25,
                    verbose=1,
                    validation_data=(X_val, y_val))

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 40s - loss: 2.3076 - acc: 0.1002 - val_loss: 2.3054 - val_acc: 0.1046
Epoch 2/25
40000/40000 [==============================] - 40s - loss: 2.3069 - acc: 0.1015 - val_loss: 2.3078 - val_acc: 0.1029
Epoch 3/25
40000/40000 [==============================] - 40s - loss: 2.3074 - acc: 0.0989 - val_loss: 2.3132 - val_acc: 0.0960
Epoch 4/25
40000/40000 [==============================] - 41s - loss: 2.3074 - acc: 0.1007 - val_loss: 2.3029 - val_acc: 0.1008
Epoch 5/25
14490/40000 [=========>....................] - ETA: 25s - loss: 2.3074 - acc: 0.0986

In [19]:
score3 = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

Test loss: 2.30294412384
Test accuracy: 0.1


In [16]:
#Model 5
from keras.constraints import maxnorm
epochs = 28
lrate = 0.014
decay = lrate/epochs
sgd_4 = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model_4=Sequential()
model_4.add(Dense(1024, activation='relu', input_shape=(3072,),kernel_constraint=maxnorm(3)))
#model_j.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model_4.add(Dropout(0.2))
model_4.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model_4.add(Dropout(0.2))
model_4.add(Dense(10, activation='softmax'))

In [17]:
model_4.compile(loss='categorical_crossentropy', optimizer=sgd_4, metrics=['accuracy'])
print(model_4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              3146752   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 3,676,682
Trainable params: 3,676,682
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
print(model_4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              3146752   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 3,676,682
Trainable params: 3,676,682
Non-trainable params: 0
_________________________________________________________________
None


In [21]:

# Fit the model
#model_j.fit(X_trn, y_trn, validation_data=(X_val, y_val), epochs=epochs, batch_size=32)
model_4.fit(x_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/28
50000/50000 [==============================] - 52s - loss: 1.2683 - acc: 0.5464 - val_loss: 1.1830 - val_acc: 0.5854
Epoch 2/28
50000/50000 [==============================] - 53s - loss: 1.2617 - acc: 0.5491 - val_loss: 1.1821 - val_acc: 0.5880
Epoch 3/28
50000/50000 [==============================] - 52s - loss: 1.2566 - acc: 0.5497 - val_loss: 1.1582 - val_acc: 0.5945
Epoch 4/28
50000/50000 [==============================] - 53s - loss: 1.2509 - acc: 0.5529 - val_loss: 1.1601 - val_acc: 0.5932
Epoch 5/28
50000/50000 [==============================] - 53s - loss: 1.2452 - acc: 0.5544 - val_loss: 1.1482 - val_acc: 0.5953
Epoch 6/28
50000/50000 [==============================] - 52s - loss: 1.2428 - acc: 0.5558 - val_loss: 1.1351 - val_acc: 0.6026
Epoch 7/28
50000/50000 [==============================] - 52s - loss: 1.2321 - acc: 0.5620 - val_loss: 1.1372 - val_acc: 0.6009
Epoch 8/28
50000/50000 [==============================

In [22]:
# Final evaluation of the model
scores = model_4.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 55.28%
